# **Finetuning Longformer Encoder-Decoder (LED) Model** 
###Trained on Multi-News Dataset
#### Adapted from Source: https://colab.research.google.com/drive/12LjJazBl7Gam0XBPy_y0CTOJZeZ34c2v?usp=sharing

## Installing packages

In [ ]:
# Checking for Sufficient RAM
!nvidia-smi

Tue Nov  1 12:01:17 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P8    12W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Installing packages
%%capture
#!pip install datasets==1.2.1
!pip install transformers==4.21.3
!pip install rouge_score
!pip install dill==0.3.4

In [ ]:
!pip install huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install git+https://github.com/huggingface/datasets.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/datasets.git to /tmp/pip-req-build-mgozp39g
  Running command git clone -q https://github.com/huggingface/datasets.git /tmp/pip-req-build-mgozp39g
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Using cached dill-0.3.6-py3-none-any.whl (110 kB)
  Attempting uninstall: dill
    Found existing installation: dill 0.3.4
    Uninstalling dill-0.3.4:
      Successfully uninstalled dill-0.3.4


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token


## Loading data

In [ ]:
# Loading datasets
from datasets import load_dataset, load_metric
train_dataset = load_dataset("multi_news", download_mode="force_redownload", split = "train")

Generating train split:   0%|          | 0/44972 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5622 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5622 [00:00<?, ? examples/s]

Dataset multi_news downloaded and prepared to /root/.cache/huggingface/datasets/multi_news/default/1.0.0/2f1f69a2bedc8ad1c5d8ae5148e4755ee7095f465c1c01ae8f85454342065a72. Subsequent calls will reuse this data.


In [ ]:
val_dataset = load_dataset("multi_news", download_mode="force_redownload", split = "validation")

Generating train split:   0%|          | 0/44972 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5622 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5622 [00:00<?, ? examples/s]

Dataset multi_news downloaded and prepared to /root/.cache/huggingface/datasets/multi_news/default/1.0.0/2f1f69a2bedc8ad1c5d8ae5148e4755ee7095f465c1c01ae8f85454342065a72. Subsequent calls will reuse this data.


In [ ]:
train_dataset

Dataset({
    features: ['document', 'summary'],
    num_rows: 44972
})

In [ ]:
val_dataset

Dataset({
    features: ['document', 'summary'],
    num_rows: 5622
})

In [ ]:
val_dataset.features

{'document': Value(dtype='string', id=None),
 'summary': Value(dtype='string', id=None)}

In [ ]:
# Looking at some data samples
train_dataset['document'][2]

'GAITHERSBURG, Md. (AP) — A small, private jet has crashed into a house in Maryland\'s Montgomery County on Monday, killing at least three people on board, authorities said. \n \n Preliminary information indicates at least three people were on board and didn\'t survive the Monday crash into home in Gaithersburg, a Washington, D.C. suburb, said Pete Piringer, a Montgomery County Fire and Rescue spokesman. \n \n He said a fourth person may have been aboard. \n \n Piringer said the jet crashed into one home around 11 a.m., setting it and two others on fire. Crews had the fire under control within an hour and were searching for anyone who may have been in the homes. \n \n Television news footage of the scene showed one home nearly destroyed, with a car in the driveway. Witnesses told television news crews that they saw the airplane appear to struggle to maintain altitude before going into a nosedive and crashing. \n \n An FAA spokesman said preliminary information shows the Embraer EMB-500

In [ ]:
train_dataset['summary'][2]

'– A twin-engine Embraer jet that the FAA describes as "on approach to Runway 14" at the Montgomery County Airpark in Gaithersburg, Maryland, crashed into a home this morning, engulfing that home in flames and setting two others on fire. Three people are dead, but the count could grow. A Montgomery County Fire rep says three fliers were killed in the crash, but notes the corporate plane may have had a fourth person on board, reports the AP. A relative of the owner of the home that was hit tells WUSA 9 that a mother with three children pre-school age and under should have been home at the time; there\'s no word on the family\'s whereabouts. The crash occurred around 11am on Drop Forge Lane, and the fire was extinguished within an hour. Crews are now searching the wreckage. A witness noted the plane appeared to "wobble" before the crash; the airport is no more than 3/4 mile from the crash scene. NTSB and FAA will investigate.'

## Data preprocessing

In [ ]:
from transformers import AutoTokenizer

In [ ]:
# Loading LED base instead of LED large model as it requires less GPU RAM to run.
tokenizer = AutoTokenizer.from_pretrained("allenai/led-base-16384")

In [ ]:
# Max input length is set to the median length of SGH articles.  Datasets will be tokenized to max_input_length.
max_input_length = 3072
max_output_length = 512
batch_size = 4

In [ ]:
# Create function to tokenize inputs and labels, create attention and global attention masks.
def process_data_to_model_inputs(batch):
    # tokenize the inputs and labels
    inputs = tokenizer(
        batch["document"],
        padding="max_length",
        truncation=True,
        max_length=max_input_length,
    )
    outputs = tokenizer(
        batch["summary"],
        padding="max_length",
        truncation=True,
        max_length=max_output_length,
    )

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask

    # create 0 global_attention_mask lists
    batch["global_attention_mask"] = len(batch["input_ids"]) * [
        [0 for _ in range(len(batch["input_ids"][0]))]
    ]

    # since above lists are references, the following line changes the 0 index for all samples
    batch["global_attention_mask"][0][0] = 1
    batch["labels"] = outputs.input_ids

    # We have to make sure that the PAD token is ignored
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in labels]
        for labels in batch["labels"]
    ]

    return batch

In [ ]:
# Select subset of training and validation data as MultiNews dataset is very large.
train_dataset = train_dataset.select(range(1000))
val_dataset = val_dataset.select(range(100))

In [ ]:
# Preprocessing training data
train_dataset = train_dataset.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=['document', 'summary'],
)

  0%|          | 0/250 [00:00<?, ?ba/s]

In [ ]:
# Preprocessing validation data
val_dataset = val_dataset.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=["document", "summary"],
)

  0%|          | 0/25 [00:00<?, ?ba/s]

In [ ]:
# Convert datasets to PyTorch format
train_dataset.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "global_attention_mask", "labels"],
)
val_dataset.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "global_attention_mask", "labels"],
)

In [ ]:
train_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'global_attention_mask', 'labels'],
    num_rows: 1000
})

In [ ]:
val_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'global_attention_mask', 'labels'],
    num_rows: 100
})

## Loading model and metrics

In [ ]:
# Load model via AutoModelForSeq2SeqLM class
from transformers import AutoModelForSeq2SeqLM
led = AutoModelForSeq2SeqLM.from_pretrained("allenai/led-base-16384", gradient_checkpointing=True, use_cache=False)

In [ ]:
# Set hyperparameters
led.config.num_beams = 2
led.config.max_length = 512
led.config.min_length = 100
led.config.length_penalty = 2.0
led.config.early_stopping = True
led.config.no_repeat_ngram_size = 3

In [ ]:
# Load rouge metric
rouge = load_metric("rouge")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  """Entry point for launching an IPython kernel.


In [ ]:
# Defining compute_metrics function
# Compute metrics function expects output pred.predictions and label pred.label_ids
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions
    
    # Decoding tokens
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    # Computing rouge score
    rouge1_output = rouge.compute(
        predictions=pred_str, references=label_str, rouge_types=["rouge1"])["rouge1"].mid
    rouge2_output = rouge.compute(
        predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid        
    rougeL_output = rouge.compute(
        predictions=pred_str, references=label_str, rouge_types=["rougeL"])["rougeL"].mid 
    rougeLsum_output = rouge.compute(
        predictions=pred_str, references=label_str, rouge_types=["rougeLsum"])["rougeLsum"].mid 

    return {
        "rouge1_precision": round(rouge1_output.precision, 4),
        "rouge1_recall": round(rouge1_output.recall, 4),
        "rouge1_fmeasure": round(rouge1_output.fmeasure, 4),
        "rouge2_precision": round(rouge2_output.precision, 4),
        "rouge2_recall": round(rouge2_output.recall, 4),
        "rouge2_fmeasure": round(rouge2_output.fmeasure, 4),
        "rougeL_precision": round(rougeL_output.precision, 4),
        "rougeL_recall": round(rougeL_output.recall, 4),
        "rougeL_fmeasure": round(rougeL_output.fmeasure, 4),
        "rougeLsum_precision": round(rougeLsum_output.precision, 4),
        "rougeLsum_recall": round(rougeLsum_output.recall, 4),
        "rougeLsum_fmeasure": round(rougeLsum_output.fmeasure, 4),
    }

## Model training, evaluation and deployment

In [ ]:
# Importing relevant packages
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from huggingface_hub.keras_mixin import push_to_hub_keras
from transformers import Trainer

In [ ]:
# Defining training arguments
training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    evaluation_strategy="steps",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    fp16=True,
    output_dir="summarise_v9",
    logging_steps=10,
    eval_steps=10,
    save_steps=10,
    save_total_limit=2,
    gradient_accumulation_steps=4,
    num_train_epochs=2,
    push_to_hub=True
)

In [ ]:
# Pass training arguments, model, tokenizer, datasets and compute_metrics function to the trainer
trainer = Seq2SeqTrainer(
    model=led,
    tokenizer=tokenizer,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

/content/summarise_v9 is already a clone of https://huggingface.co/debbiesoon/summarise_v9. Make sure you pull the latest changes with `repo.git_pull()`.
Using cuda_amp half precision backend


In [ ]:
# Train model, generate evaluation metrics
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1000
  Num Epochs = 2
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 4
  Total optimization steps = 124


Step,Training Loss,Validation Loss,Rouge1 Precision,Rouge1 Recall,Rouge1 Fmeasure,Rouge2 Precision,Rouge2 Recall,Rouge2 Fmeasure,Rougel Precision,Rougel Recall,Rougel Fmeasure,Rougelsum Precision,Rougelsum Recall,Rougelsum Fmeasure
10,2.809500,2.539277,0.287000,0.535800,0.367400,0.102300,0.191700,0.131100,0.137400,0.261500,0.177100,0.137400,0.261500,0.177100
20,2.605600,2.475246,0.500500,0.326400,0.381100,0.166300,0.105400,0.124900,0.258200,0.166700,0.195700,0.258200,0.166700,0.195700
30,2.594300,2.442224,0.461500,0.383300,0.404700,0.147300,0.127300,0.132100,0.224200,0.188500,0.198100,0.224200,0.188500,0.198100
40,2.484200,2.418639,0.467500,0.382900,0.408100,0.158100,0.129400,0.138400,0.228600,0.187000,0.199500,0.228600,0.187000,0.199500
50,2.445400,2.399027,0.467000,0.408000,0.422200,0.163300,0.142900,0.147700,0.229400,0.200800,0.207600,0.229400,0.200800,0.207600
60,2.362200,2.385656,0.456700,0.389800,0.410000,0.143300,0.123300,0.129500,0.220500,0.187600,0.197600,0.220500,0.187600,0.197600
70,2.403400,2.383516,0.451500,0.430400,0.429400,0.152600,0.147900,0.145900,0.218300,0.209000,0.207800,0.218300,0.209000,0.207800
80,2.261200,2.380399,0.455000,0.419300,0.423600,0.151800,0.142900,0.142700,0.217700,0.202500,0.203700,0.217700,0.202500,0.203700
90,2.256300,2.376820,0.482100,0.391000,0.419600,0.165200,0.135700,0.144000,0.238500,0.192900,0.206900,0.238500,0.192900,0.206900
100,2.243000,2.376791,0.454600,0.409300,0.416100,0.155200,0.140200,0.142200,0.224800,0.201600,0.205200,0.224800,0.201600,0.205200


***** Running Evaluation *****
  Num examples = 100
  Batch size = 4
Saving model checkpoint to summarise_v9/checkpoint-10
Configuration saved in summarise_v9/checkpoint-10/config.json
Model weights saved in summarise_v9/checkpoint-10/pytorch_model.bin
tokenizer config file saved in summarise_v9/checkpoint-10/tokenizer_config.json
Special tokens file saved in summarise_v9/checkpoint-10/special_tokens_map.json
tokenizer config file saved in summarise_v9/tokenizer_config.json
Special tokens file saved in summarise_v9/special_tokens_map.json
Several commits (3) will be pushed upstream.
Deleting older checkpoint [summarise_v9/checkpoint-50] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 100
  Batch size = 4
Saving model checkpoint to summarise_v9/checkpoint-20
Configuration saved in summarise_v9/checkpoint-20/config.json
Model weights saved in summarise_v9/checkpoint-20/pytorch_model.bin
tokenizer config file saved in summarise_v9/checkpoint-20/tokenizer_confi

TrainOutput(global_step=124, training_loss=2.4090794670966362, metrics={'train_runtime': 27887.6102, 'train_samples_per_second': 0.072, 'train_steps_per_second': 0.004, 'total_flos': 4034106023215104.0, 'train_loss': 2.4090794670966362, 'epoch': 1.99})

This completes the fine-tuning tutorial for LED. This training script with some small changes was used to train [this](https://huggingface.co/patrickvonplaten/led-large-16384-pubmed) checkpoint, called `" patrickvonplaten/led-large-16384-pubmed"` on a single GPU for ca. 3 days. Evaluating `" patrickvonplaten/led-large-16384-pubmed"` on Pubmed's test data gives a Rouge-2 score of **19.33** which is around 1 Rouge-2 point below SOTA performance on Pubmed.

In the Appendix below, the condensed training and evaluation scripts that were used locally to finetune `" patrickvonplaten/led-large-16384-pubmed"` are attached.

In [ ]:
# Push model to HuggingFace Hub
trainer.push_to_hub(commit_message="Training complete")

Saving model checkpoint to summarise_v9
Configuration saved in summarise_v9/config.json
Model weights saved in summarise_v9/pytorch_model.bin
tokenizer config file saved in summarise_v9/tokenizer_config.json
Special tokens file saved in summarise_v9/special_tokens_map.json
Several commits (4) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.34k/618M [00:00<?, ?B/s]

Upload file runs/Nov01_12-04-23_a2fe2008fc1c/events.out.tfevents.1667304331.a2fe2008fc1c.5798.0:  19%|#8      …

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/debbiesoon/summarise_v9
   a518aa1..a136484  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/debbiesoon/summarise_v9
   a518aa1..a136484  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Sequence-to-sequence Language Modeling', 'type': 'text2text-generation'}, 'dataset': {'name': 'multi_news', 'type': 'multi_news', 'config': 'default', 'split': 'train', 'args': 'default'}}
To https://huggingface.co/debbiesoon/summarise_v9
   a136484..6e18240  main -> main

   a136484..6e18240  main -> main



'https://huggingface.co/debbiesoon/summarise_v9/commit/a136484eecafff4c4970146b31c1431a1d815fc5'

In [ ]:
# Save model to Google Drive
trainer.save_model('/content/drive/MyDrive/SGH Project/summarize_v9/')

Saving model checkpoint to /content/drive/MyDrive/SGH Project/summarize_v9/
Configuration saved in /content/drive/MyDrive/SGH Project/summarize_v9/config.json
Model weights saved in /content/drive/MyDrive/SGH Project/summarize_v9/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/SGH Project/summarize_v9/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/SGH Project/summarize_v9/special_tokens_map.json
Saving model checkpoint to summarise_v9
Configuration saved in summarise_v9/config.json
Model weights saved in summarise_v9/pytorch_model.bin
tokenizer config file saved in summarise_v9/tokenizer_config.json
Special tokens file saved in summarise_v9/special_tokens_map.json
Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Sequence-to-sequence Language Modeling', 'type': 'text2text-generation'}, 'dataset': {'name': 'multi_news', 'type': 'multi_news', 'config': 'default', 'split': 'train', 'args': 'def